# Info
* We currently have a de-normalised dataset of Orders in "Kelly's Dataset.csv"
* This is difficult to visualize and query for if we load it as is onto Ne04J
* So, the following code blocks helps us create normalised datasets (CSVs) which can be linked to the original `Kelly's Dataset` for better results.
* **Note** : This file has to be executed in a "Google Colab Notebook"

In [2]:
pip install pandas


In [3]:
from google.colab import files
uploaded = files.upload()

Saving Kelly's Dataset.csv to Kelly's Dataset.csv


# Generate Normalised Datasets
* After analysing the dataset, figured out that splitting it into the following 4 different CSV files helped
  - **products.csv** : All the ProductIDs along with Product Categories
  - **stores.csv** : All the StoreIDs along with Countries
  - **product_categories.csv** : All the Product Categories (Mens/Womens). A neo4j relation from `products.csv` will be created to this.
  - **countries.csv** : All the Countries (Spain/Germany/UK/Italy/France). A neo4j relation from `stores.csv` will be created to this.
* `Kelly's Dataset` will be linked to `products` and `stores`


In [11]:
import pandas as pd

# Read the original CSV file
df = pd.read_csv("Kelly's Dataset.csv")

# Get unique Products with Categories
unique_products = df[['Product ID', 'Product Category']].drop_duplicates()
print("Unique Products: ", unique_products.shape[0])
unique_products.to_csv('products.csv', index=False)

# Get unique Stores
unique_stores = df[['Store ID', 'Country']].drop_duplicates()
print("Unique Stores: ", unique_stores.shape[0])
unique_stores.to_csv('stores.csv', index=False)

# Get unique Contries
unique_countries = pd.DataFrame({'Country': df['Country'].unique()})
print("Unique Countries: ", unique_countries.shape[0])
unique_countries.to_csv('countries.csv', index=False)

# Get unique Product Categories
unique_product_categories = pd.DataFrame({'Product Category': df['Product Category'].unique()})
print("Unique Product Categories: ", unique_product_categories.shape[0])
unique_product_categories.to_csv('product_categories.csv', index=False)



Unique Products:  1831
Unique Stores:  995
Unique Countries:  5
Unique Product Categories:  2


# Import Datasets into Neo4J instance
![Neo4J Relations](./images/neo4j-relations.png)

* All the generated CSVs are placed in `neo4j/kellys-data-importer` folder.
* Zip this folder and import directly into Neo4J instance using **Open Model (with data)** option.
* Alternatively, you can upload `neo4j_importer_model.json` individually using **Open Model** option.
  - Upload `csv` files using **Browse** option in Files pane

![Import Data](./images/import-data.png)

# Visualisation
If everything goes well, you should be able to see the relations like this in the "Explore" tab

![Graph](./images/graph-exploration.png)

# Sample Queries

## Query to sum all the units of a Product
```
MATCH (store:Kellys)-[:CONTAINS]->(product:Product)
WHERE product.`Product ID` = "18702"
RETURN SUM(store.`Units Sold`) AS totalUnitsSold;
```

![Sum of all units sold for Product 18702](./images/sum-all-units-of-a-product.png)

## Query to get top 10 highest selling Products
```
MATCH (store:Kellys)-[:CONTAINS]->(product:Product)
WITH product.`Product ID` AS productId, SUM(store.`Units Sold`) AS totalUnitsSold
RETURN productId, totalUnitsSold
ORDER BY totalUnitsSold DESC
LIMIT 10;
```

![Top 10 highest selling products](./images/top-10-highest-selling-products.png)